### Model Description:
- LSA + TF-IDF (LSA is is SVD on doc-term matrix)
- Create function to create batches of doc intrusion
- This version removes duplicate tweets
- This version randomizes the order of the intruders in the output

In [1]:
import pandas as pd
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize, RegexpTokenizer
from nltk.util import ngrams
nltk.download('stopwords')
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
import re
import numpy as np
import random

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jennihawk/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
#got tweets at 1:15 Pacific Time 10/14/22
tweets = pd.read_csv('/Users/jennihawk/Documents/Data Science Projects/Project_NLP/TweetBatch3.csv')
tweets

,text,cleaned
0,@ReallyAmerican1 #Roevember and\n#ForThePeople...,roevember and forthepeople and votebluein2022...
1,RT @sandibachom: IS THIS THING ON???!!This is ...,rt is this thing on this is pathetic acting se...
2,RT @sandibachom: IS THIS THING ON???!!This is ...,rt is this thing on this is pathetic acting se...
3,RT @tleehumphrey: Today is the beginning of th...,rt today is the beginning of the inquiry into ...
4,RT @AdamKinzinger: Mitch McConnell.\nKevin McC...,rt mitch mcconnell kevin mccarthy they both kn...
...,...,...
34988,RT @Adrian_Fontes: The January 6th committee j...,rt fontes the january 6th committee just concl...
34989,#January6thCommitteeHearings and everyone runn...,january6thcommitteehearings and everyone runn...
34990,RT @sandibachom: IS THIS THING ON???!!This is ...,rt is this thing on this is pathetic acting se...
34991,So they are gonna subpoena Trump I am guessing...,so they are gonna subpoena trump am guessing t...


### Remove stop words from cleaned column

In [3]:
stop_words = stopwords.words('english')
stop_words.extend(['rt', 'january', 'january6thcommitteehearings', 'ja', 'january6thcomm'])
tweets['cleaned'] = tweets['cleaned'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))

### Tokenize Words in Cleaned Column

In [4]:
tweets['tokenized'] = tweets.apply(lambda row: nltk.word_tokenize(row['cleaned']), axis=1)

In [5]:
tweets.head()

,text,cleaned,tokenized
0,@ReallyAmerican1 #Roevember and\n#ForThePeople...,roevember forthepeople votebluein2022 standwit...,"[roevember, forthepeople, votebluein2022, stan..."
1,RT @sandibachom: IS THIS THING ON???!!This is ...,thing pathetic acting sec defense chris miller...,"[thing, pathetic, acting, sec, defense, chris,..."
2,RT @sandibachom: IS THIS THING ON???!!This is ...,thing pathetic acting sec defense chris miller...,"[thing, pathetic, acting, sec, defense, chris,..."
3,RT @tleehumphrey: Today is the beginning of th...,today beginning inquiry trudeau gov use emerge...,"[today, beginning, inquiry, trudeau, gov, use,..."
4,RT @AdamKinzinger: Mitch McConnell.\nKevin McC...,mitch mcconnell kevin mccarthy knew trump resp...,"[mitch, mcconnell, kevin, mccarthy, knew, trum..."


In [6]:
corpus = list(tweets.cleaned)

In [7]:
# turn corpus into a document-term matrix
vectorizer = CountVectorizer()
doc_term = vectorizer.fit_transform(corpus)

## LSA + TF-IDF 

### TF-IDF Vectorizer
- Instead of counts filling up matrix each value will be a TF-IDF value which is the term frequency or the percent of the document that consists of a particular term times the inverse document frequency, which is how rare the term is. 

In [8]:
#get the doc term matrix populated with the TF-IDF scores for the text corpus.
tfidf_vect = TfidfVectorizer()
tf_doc_term = tfidf_vect.fit_transform(corpus).toarray()
tf_doc_term_df = pd.DataFrame(tf_doc_term, columns = tfidf_vect.get_feature_names_out())

In [9]:
tf_doc_term.shape

(34993, 9399)

In [10]:
pd.set_option('display.max_columns', None)

In [11]:
#tf_doc_term_df.iloc[:, :300]

### LSA: Reduce Dimensionality 
- LSA is just SVD on the document term matrix

In [12]:
#number of topics indicated in arg of TruncatedSVD()
lsa2 = TruncatedSVD(n_components = 4, random_state = 122)
lsa2.fit(tf_doc_term)
lsa2

TruncatedSVD(n_components=4, random_state=122)

In [13]:
components=lsa2.components_
vocab = tfidf_vect.get_feature_names_out()

In [14]:
tf_topic_term = lsa2.components_.round(3)
tf_topic_term

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

#### Turn topic_term into dataframe

In [15]:
tf_topic_term_df = pd.DataFrame(tf_topic_term.round(3),
                index = ["component_1", "component_2", "component_3", "component_4"],
                columns = tfidf_vect.get_feature_names_out())
tf_topic_term_df

,00,000,01,05,08,09,10,100,1000,1000x,100s,100yds,101,102,10am,10h,10percenttothebigguy,10pm,10th,11,118,1199,11thhour,12,12deoctubre,12h,13,139,13th,14,147,14a,14th,14x,15,150,154,157,15s,16,17,1776,18,180,182,1857,187,187minutes,19,1961,1973,1980,1984,1a,1pm,1st,20,2007,2008,2011,2015,2016,2017,2018,2019,202,2020,2021,2022,2022midterms,2023,2024,2030,20th,21,215,216,22,23,23pm,24,240,246,25,256,25th,26,261,271,28,29,292,29m,2a,2d,2day,2ndwin,2pro,30,30k,31,31st,32,33,350,360,37thcircuitcourt,38,3am,3d,3rd,3x,40,401k,4092,440,44023,45,45th,45yrs,47,49,4house,4s,4senate,4th,50,50th,52,55,551,56,584,5p,5pm,5t,5th,60,6000,61,62,627,628,645,69,6pm,6t,6th,70,700,703,714,7202,7208,7394,74,75,76,78,780,79,7pm,7th,80,800th,80s,81,82,82years,85,871,88,8pm,8th,90,93,95,957,99,9th,aaaaaand,aaliyah,aaron,ab,aback,abandon,abandoned,abandonment,abc,abc7chicago,abdicated,abducted,abetted,abetting,abhorrent,abiding,ability,abject,able,abo,abolish,abolishtheelectoralcollege,abortion,abortionist,abou,abraham,abrams,absence,absentee,abso,absofuckinglutely,absolute,absolutely,absolve,abt,abundantly,abuse,abuser,abysmal,ac,ac360,accept,acceptable,accepted,access,accessible,accomplices,accomplished,accomplishing,accomplishment,accord,according,accoubtable,account,accountabi,accountability,accountabilitynow,accountabl,accountable,accounting,accounts,acct,accurate,accuse,accused,accusers,accuses,accusing,ace,achieve,acknowledge,acknowledged,acknowledgement,acknowledges,acknowledging,acmche,acolyte,acquiesce,across,act,acted,acting,action,actions,activate,active,actively,activist,activity,actor,actors,acts,actu,actual,actually,ad,adage,adam,adamkinzinger,adams,adamschiff,add,added,addict,addicted,addicts,addition,additional,address,addressed,addresses,addressing,adequate,adjourned,admin,administration,admirably,admire,admissible,admission,admissions,admit,admits,admitted,adolph,adopted,ados,ads,adult,adults,advance,advanced,advantage,adversary,advertising,advice,advisers,advising,advisor,advisors,advocate,aekvfb,aew,aewdynamite,af,afewgoodmen,affair,affiliate,affiliation,affilliatelink,affirmative,afford,afghanistan,afraid,africa,afrobeats,afternoon,afterthought,afterwards,afterwhich,ag,aga,age,agencies,agency,agenda,agent,agentorange,agents,ago,agree,agreed,agreeing,agrees,aguilar,ah,aha,ahead,aid,aide,aided,aides,aiding,aimed,aint,air,airbnbs,aired,airing,airport,airs,ak,aka,akin,al,alacrity,alarmed,alaska,albuquerque,alcohol,alcoholic,alds,alert,alerts,alesandro,alessandro,alex,alexandra,alexandrapelosi,alexandriapelosi,alexis,alexjonesscumbucket,alexjoneswasright,alexwagner,alexwagnertonight,alies,aligned,alignment,alive,allegations,alleged,allegedly,allergy,allforshow,alliance,allies,allison,allow,allowed,allowing,allstars,alltrumpsarecriminals,allyo,almighty,almost,alo,alon,alone,along,alow,alpha,alphaman,already,also,alt,alternate,alternatively,although,altuve,alum,alva,alvarez,always,alyssa,alyssafarahgriffin,ama,amalgam,amas,amaze,amazed,amazing,amazingly,amazon,amberdiceless,amcneverleaving,amen,amendment,americ,america,americafirst,americahasaproblem,american,americanhero,americanism,americans,amiright,amiss,among,amongst,amor,amount,amp,amphibian,amplification,amplified,amundialdelavisi,amusing,amwatching,amwriting,amy,ana,anal,analysis,analyst,ancestors,anchor,anchors,anderson360,andor,andrea,andreamitchell,andy,anesthetists,ang,anger,angerer,angry,anika,anil,animals,animelove,animeworld,ankitgupta,ann,annabettis,annah,annebra,anniversary,announce,announced,announcement,announcing,anointed,anonymous,another,answer,answered,answering,answers,antal,anti,anticipated,anticipation,antidemocratic,antifa,antoniogibson,anybody,anymore,anyon,anyone,anything,anytime,anyway,anyways,anywhere,aoc,apart,apartment,apatriopatriot,ape,apologist,apologize,apologizetojhope,apology,apostolic,app,appalled,appalling,apparel,apparent,apparently,appeal,appeals,appear,appearance,appeared,appearing,appears,appease,apple,applied,apply,appointed,appointee,appointee

#### Easier to read: Function displays the top terms in each topic 

In [16]:
def display_topics(model, feature_names, no_top_words, topic_names = None): 
    for ix, topic in enumerate(model.components_):
        if not topic_names or not topic_names[ix]:
            print("\nTopic ", ix + 1)
        else:
            print("\nTopic: ", topic_names[ix])
        print(", ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))
    print("\n")
    return model, feature_names, no_top_words

#### Get 8 terms for each topic

In [17]:
# number is the number of top terms you want to see
output = display_topics(lsa2, tfidf_vect.get_feature_names_out(), 8)
output


Topic  1
kevin, backed, mccarthy, mitch, mcconnell, responsible, called, knew

Topic  2
mark, democracy, violence, creating, hamill, correct, overthrowing, love

Topic  3
decided, rudy, hatch, giuliani, meadows, lost, yet, stone

Topic  4
demands, oath, unanimously, deserves, history, voted, subpoena, testify




(TruncatedSVD(n_components=4, random_state=122),
 array(['00', '000', '01', ..., 'zoom', 'zuma', 'zzy'], dtype=object),
 8)

#### Name the topics

In [18]:
display_topics(lsa2, tfidf_vect.get_feature_names_out(), 8, ['Mitch_Kevin_Knew', 'Hamill_Democracy','Meadows_Guiliani_Stone','Subpoena_Trump']);


Topic:  Mitch_Kevin_Knew
kevin, backed, mccarthy, mitch, mcconnell, responsible, called, knew

Topic:  Hamill_Democracy
mark, democracy, violence, creating, hamill, correct, overthrowing, love

Topic:  Meadows_Guiliani_Stone
decided, rudy, hatch, giuliani, meadows, lost, yet, stone

Topic:  Subpoena_Trump
demands, oath, unanimously, deserves, history, voted, subpoena, testify




#### Determine which topics are in each document. Transform the original doc_term matrix into a document-topic matrix

In [19]:
doc_topic2 = lsa2.transform(doc_term)
doc_topic2.shape

(34993, 4)

#### Turn doc_topic matrix into a dataframe

In [20]:
doc_topic_df2 = pd.DataFrame(doc_topic2.round(5), index = corpus, columns = ["Mitch_Kevin_Knew", "Hamill_Democracy","Meadows_Guiliani_Stone","Subpoena_Trump"])

In [21]:
doc_topic_df2.reset_index(inplace=True)
doc_topic_df2 = doc_topic_df2.rename(columns = {'index':'Processed_Tweets'})

In [22]:
doc_topic_df2.head()

,Processed_Tweets,Mitch_Kevin_Knew,Hamill_Democracy,Meadows_Guiliani_Stone,Subpoena_Trump
0,roevember forthepeople votebluein2022 standwit...,0.00002,0.00016,0.00011,0.00036
1,thing pathetic acting sec defense chris miller...,0.00020,0.00288,0.00909,0.00835
2,thing pathetic acting sec defense chris miller...,0.00020,0.00288,0.00909,0.00835
3,today beginning inquiry trudeau gov use emerge...,0.00026,0.00242,0.00319,0.00389
4,mitch mcconnell kevin mccarthy knew trump resp...,2.98070,0.01766,0.03322,0.05555


In [23]:
# add original tweets to df
doc_topic_df2['Tweets'] = tweets['text']

In [24]:
#list(doc_topic_df2)

In [25]:
#change order of columns
doc_topic_df2 = doc_topic_df2.reindex(['Tweets','Processed_Tweets','Mitch_Kevin_Knew','Hamill_Democracy', 'Meadows_Guiliani_Stone', 'Subpoena_Trump' ], axis=1)

In [26]:
doc_topic_df2.head()

,Tweets,Processed_Tweets,Mitch_Kevin_Knew,Hamill_Democracy,Meadows_Guiliani_Stone,Subpoena_Trump
0,@ReallyAmerican1 #Roevember and\n#ForThePeople...,roevember forthepeople votebluein2022 standwit...,0.00002,0.00016,0.00011,0.00036
1,RT @sandibachom: IS THIS THING ON???!!This is ...,thing pathetic acting sec defense chris miller...,0.00020,0.00288,0.00909,0.00835
2,RT @sandibachom: IS THIS THING ON???!!This is ...,thing pathetic acting sec defense chris miller...,0.00020,0.00288,0.00909,0.00835
3,RT @tleehumphrey: Today is the beginning of th...,today beginning inquiry trudeau gov use emerge...,0.00026,0.00242,0.00319,0.00389
4,RT @AdamKinzinger: Mitch McConnell.\nKevin McC...,mitch mcconnell kevin mccarthy knew trump resp...,2.98070,0.01766,0.03322,0.05555


#### Make new dataframe that filters values of topic columns
We're removing content with lowest scores
- Filters => 25% of total column value
- Filtering went from 34,993 tweets to 19,741

In [27]:
top_score_tweets = doc_topic_df2.copy()

In [28]:
filt1 = top_score_tweets['Mitch_Kevin_Knew'].quantile(0.25)
filt2 = top_score_tweets['Hamill_Democracy'].quantile(0.25)
filt3 = top_score_tweets['Meadows_Guiliani_Stone'].quantile(0.25)
filt4 = top_score_tweets['Subpoena_Trump'].quantile(0.25)

In [29]:
top_score_tweets = top_score_tweets[top_score_tweets['Mitch_Kevin_Knew'] >= filt1]
top_score_tweets = top_score_tweets[top_score_tweets['Hamill_Democracy'] >= filt2]
top_score_tweets = top_score_tweets[top_score_tweets['Meadows_Guiliani_Stone'] >= filt3]
top_score_tweets = top_score_tweets[top_score_tweets['Subpoena_Trump'] >= filt4]

In [30]:
top_score_tweets.head()

,Tweets,Processed_Tweets,Mitch_Kevin_Knew,Hamill_Democracy,Meadows_Guiliani_Stone,Subpoena_Trump
4,RT @AdamKinzinger: Mitch McConnell.\nKevin McC...,mitch mcconnell kevin mccarthy knew trump resp...,2.98070,0.01766,0.03322,0.05555
8,RT @2RawTooReal: I still cant get over the fac...,still cant get fact nancy pelosi sh ts wanted ...,0.24355,0.24108,0.25057,0.56588
11,RT @Navajo_6: You can’t tell me you have an ou...,6 tell ounce brains head still believe trump p...,0.23625,0.16989,0.20159,0.27820
15,RT @StandForBetter: The Secret Service clearly...,secret service clearly involved coverup trump ...,0.23554,0.17287,0.21018,0.27962
19,RT @Navajo_6: You can’t tell me you have an ou...,6 tell ounce brains head still believe trump p...,0.23625,0.16989,0.20159,0.27820


In [31]:
max_values2 = top_score_tweets[['Mitch_Kevin_Knew','Hamill_Democracy','Meadows_Guiliani_Stone', 'Subpoena_Trump']].max(axis=1)

In [32]:
max_values2

4        2.98070
8        0.56588
11       0.27820
15       0.27962
19       0.27820
          ...   
34985    1.66830
34986    0.09197
34988    0.10602
34991    0.53460
34992    0.29832
Length: 19741, dtype: float64

In [33]:
#max_values = top_score_tweets.max(axis=1)

In [34]:
#max_values

In [35]:
# Select only the numeric columns from the DataFrame
numeric_columns = top_score_tweets.select_dtypes(include=[int, float])

In [36]:
# Find the largest value in each row of the numeric columns
largest_values_per_row = numeric_columns.max(axis=1)

In [37]:
# Create a boolean mask where True indicates the largest value in each row
is_largest_numeric = numeric_columns.eq(largest_values_per_row, axis=0).idxmax(axis=1)

In [38]:
is_largest_numeric

4        Mitch_Kevin_Knew
8          Subpoena_Trump
11         Subpoena_Trump
15         Subpoena_Trump
19         Subpoena_Trump
               ...       
34985      Subpoena_Trump
34986      Subpoena_Trump
34988      Subpoena_Trump
34991      Subpoena_Trump
34992      Subpoena_Trump
Length: 19741, dtype: object

#### Adds the 'is_largest_numeric' series to the dataframe

In [39]:
top_score_tweets['topic_label']=pd.Series(is_largest_numeric)

In [40]:
top_score_tweets.head()

,Tweets,Processed_Tweets,Mitch_Kevin_Knew,Hamill_Democracy,Meadows_Guiliani_Stone,Subpoena_Trump,topic_label
4,RT @AdamKinzinger: Mitch McConnell.\nKevin McC...,mitch mcconnell kevin mccarthy knew trump resp...,2.98070,0.01766,0.03322,0.05555,Mitch_Kevin_Knew
8,RT @2RawTooReal: I still cant get over the fac...,still cant get fact nancy pelosi sh ts wanted ...,0.24355,0.24108,0.25057,0.56588,Subpoena_Trump
11,RT @Navajo_6: You can’t tell me you have an ou...,6 tell ounce brains head still believe trump p...,0.23625,0.16989,0.20159,0.27820,Subpoena_Trump
15,RT @StandForBetter: The Secret Service clearly...,secret service clearly involved coverup trump ...,0.23554,0.17287,0.21018,0.27962,Subpoena_Trump
19,RT @Navajo_6: You can’t tell me you have an ou...,6 tell ounce brains head still believe trump p...,0.23625,0.16989,0.20159,0.27820,Subpoena_Trump


In [41]:
top_score_tweets.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19741 entries, 4 to 34992
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Tweets                  19741 non-null  object 
 1   Processed_Tweets        19741 non-null  object 
 2   Mitch_Kevin_Knew        19741 non-null  float64
 3   Hamill_Democracy        19741 non-null  float64
 4   Meadows_Guiliani_Stone  19741 non-null  float64
 5   Subpoena_Trump          19741 non-null  float64
 6   topic_label             19741 non-null  object 
dtypes: float64(4), object(3)
memory usage: 1.2+ MB


In [42]:
simple_df = top_score_tweets.copy()
simple_df.drop(columns=['Mitch_Kevin_Knew', 'Hamill_Democracy', 'Meadows_Guiliani_Stone', 'Subpoena_Trump'], inplace=True)

In [43]:
simple_df.head()

,Tweets,Processed_Tweets,topic_label
4,RT @AdamKinzinger: Mitch McConnell.\nKevin McC...,mitch mcconnell kevin mccarthy knew trump resp...,Mitch_Kevin_Knew
8,RT @2RawTooReal: I still cant get over the fac...,still cant get fact nancy pelosi sh ts wanted ...,Subpoena_Trump
11,RT @Navajo_6: You can’t tell me you have an ou...,6 tell ounce brains head still believe trump p...,Subpoena_Trump
15,RT @StandForBetter: The Secret Service clearly...,secret service clearly involved coverup trump ...,Subpoena_Trump
19,RT @Navajo_6: You can’t tell me you have an ou...,6 tell ounce brains head still believe trump p...,Subpoena_Trump


In [44]:
simple_df.shape

(19741, 3)

In [45]:
remove_dupes = simple_df.copy()

In [46]:
remove_dupes.drop_duplicates(subset='Tweets', inplace=True)

In [47]:
remove_dupes.drop_duplicates(subset='Processed_Tweets', inplace=True)

In [48]:
remove_dupes.shape

(2043, 3)

In [49]:
remove_dupes.head(100)

,Tweets,Processed_Tweets,topic_label
4,RT @AdamKinzinger: Mitch McConnell.\nKevin McC...,mitch mcconnell kevin mccarthy knew trump resp...,Mitch_Kevin_Knew
8,RT @2RawTooReal: I still cant get over the fac...,still cant get fact nancy pelosi sh ts wanted ...,Subpoena_Trump
11,RT @Navajo_6: You can’t tell me you have an ou...,6 tell ounce brains head still believe trump p...,Subpoena_Trump
15,RT @StandForBetter: The Secret Service clearly...,secret service clearly involved coverup trump ...,Subpoena_Trump
24,RT @ifindkarma: “Trump’s claim that Pelosi fai...,trump claim pelosi failed call law enforcement...,Meadows_Guiliani_Stone
...,...,...,...
732,@BradMossEsq It wasn’t even willful blindness....,even willful blindness since 9 11 law enforcem...,Subpoena_Trump
737,RT @ifindkarma: “[Trump reacting to the Suprem...,trump reacting supreme court rejecting electio...,Subpoena_Trump
745,And here we have House Speaker Nancy Pelosi sh...,house speaker nancy pelosi showing concern vp ...,Subpoena_Trump
749,RT @GoodTroubleShow: We are Livestreaming the ...,livestreaming 6th hearing youtube 1pm eastern ...,Subpoena_Trump


## Batch Work New Flow
- To make this work the improvements: 
    - pull out topic as a keyword because you want check intrusions for each topic. Instead of random choice make it an arg. 
- video 1:01:05 improve the code by mixing the order of the dictionaries in the batch so not so easy to predict. Currently intruder always last
- Input: Pandas Dataframe
- Output: List of Dictionaries
- -> List[Dict[str, str]]
- Expected_result = [{“document”: doc1, “topic”: topic 1, “intruder”: 0}, {“document”: doc2, “topic”: topic 1, “intruder”: 0}, {“document”: doc3, “topic”: topic 1, “intruder”: 0}, {“document”: doc4, “topic”: topic 1, “intruder”: 0}, {“document”: doc120, “topic”: NOT topic 1, “intruder”: 1}]

In [123]:
def create_batch(df, main_topic, specific_topic):
    # Get unique topics
    topics = df['topic_label'].unique()

    # Check if the main_topic is in the topics
    if main_topic not in topics:
        raise ValueError(f"Main topic '{main_topic}' not found in the DataFrame.")
    
    batch_topic_df = df[df['topic_label'] == main_topic]
    batch = batch_topic_df.sample(n=4).to_dict('records')
    
    # Mark these documents as not intruders
    for doc in batch:
        doc['intruder'] = 0

    # Select intruder documents from another topic
    intruder_df = df[df['topic_label'] == specific_topic].sample(1)

    # Converting ths sampled 'intruder_df' into a list of dicts and marking as intruders
    intruders = intruder_df.to_dict('records')  
    for intruder in intruders:
        intruder['intruder'] = 1

    # Add the intruder document to the batch
    batch.extend(intruders)
    
    # Shuffle the batch to randomly position the intruder
    random.shuffle(batch)

    return batch


### Run the function on different topics

In [124]:
#topics
remove_dupes.topic_label.unique()

array(['Mitch_Kevin_Knew', 'Subpoena_Trump', 'Meadows_Guiliani_Stone',
       'Hamill_Democracy'], dtype=object)

#### Batch 1 Mitch Kevin Knew / Intruder 1

In [125]:
#creates list of dicts
mitch_kev_1 = create_batch(remove_dupes, 'Mitch_Kevin_Knew', 'Hamill_Democracy')
#creates dataframe from list of dicts
mitch_kev_1 = pd.DataFrame.from_records(mitch_kev_1)

In [126]:
mitch_kev_1

,Tweets,Processed_Tweets,topic_label,intruder
0,@The_FJC That #January6thCommitteeHearings was...,fjc planned yesterday exactly reason knew repo...,Mitch_Kevin_Knew,0
1,@washingtonpost #America #AmericaFirst #Januar...,america americafirst making martyr trump maga,Hamill_Democracy,1
2,Donald Trump knew he lost.” 2/2 - Chmn. Bennie...,donald trump knew lost 2 2 chmn bennie thompso...,Mitch_Kevin_Knew,0
3,These #January6thCommitteeHearings have been e...,explosive knew jan 6 premeditated tfg amp team...,Mitch_Kevin_Knew,0
4,RT @ColorMeStevie: Mr. Trump is responsible fo...,mr trump responsible 6th period end lock,Mitch_Kevin_Knew,0


#### Batch 1 Mitch Kevin Knew / Intruder 2

In [127]:
#creates list of dicts
mitch_kev_2 = create_batch(remove_dupes, 'Mitch_Kevin_Knew', 'Subpoena_Trump')
mitch_kev_2 = pd.DataFrame.from_records(mitch_kev_2)

In [128]:
mitch_kev_2

,Tweets,Processed_Tweets,topic_label,intruder
0,And here is weasel Mitch McConnell …in April 2...,weasel mitch mcconnell april 2022 said would s...,Mitch_Kevin_Knew,0
1,On Jan 6 speaker Pelosi was President pelosi s...,jan 6 speaker pelosi president pelosi one trul...,Subpoena_Trump,1
2,RT @taradublinrocks: Trump knew he lost. \nTru...,trump knew lost trump team knew lost kept goin...,Mitch_Kevin_Knew,0
3,The committee presented all the evidence. Trum...,committee presented evidence trump lap dogs st...,Mitch_Kevin_Knew,0
4,FBI Director Chris Wray IS PART of the #Republ...,fbi director chris wray part republicans coup ...,Mitch_Kevin_Knew,0


#### Batch 1 Mitch Kevin Knew / Intruder 3

In [129]:
mitch_kev_3 = create_batch(remove_dupes, 'Mitch_Kevin_Knew', 'Meadows_Guiliani_Stone')
mitch_kev_3 = pd.DataFrame.from_records(mitch_kev_3)

In [130]:
mitch_kev_3

,Tweets,Processed_Tweets,topic_label,intruder
0,@djrothkopf Why is this a surprise when we alr...,surprise already knew white supremacists domes...,Mitch_Kevin_Knew,0
1,Trump is responsible for the insurrection and ...,trump responsible insurrection finally must te...,Mitch_Kevin_Knew,0
2,RT @dudeonthebay: Trump to McCarthy - “They’re...,trump mccarthy people antifa mccarthy sir peop...,Mitch_Kevin_Knew,0
3,“DJT knew he lost. And he did everything he c...,djt knew lost everything could prevent certifi...,Mitch_Kevin_Knew,0
4,RT @argus_constant: @Justme10000000 @waltshaub...,constant trump defender excuses knowing lost d...,Meadows_Guiliani_Stone,1


#### Batch 2 Hamill_Democracy / Intruder 1

In [131]:
hamill_1 = create_batch(remove_dupes, 'Hamill_Democracy', 'Mitch_Kevin_Knew')
hamill_1 = pd.DataFrame.from_records(hamill_1)

In [132]:
hamill_1

,Tweets,Processed_Tweets,topic_label,intruder
0,@robert_burnier And Biden managed to do all th...,burnier biden managed fraud trump charge count...,Hamill_Democracy,0
1,Chilling video with the #January6thCommitteeHe...,chilling video today proves joshhawley cut amp...,Hamill_Democracy,0
2,“DJT knew he lost. And he did everything he c...,djt knew lost everything could prevent certifi...,Mitch_Kevin_Knew,1
3,RT @GerberKawasaki: This is insane footage of ...,insane footage attack democracy desperation he...,Hamill_Democracy,0
4,@SpeakerPelosi was calm and getting organized ...,calm getting organized like queen control mean...,Hamill_Democracy,0


#### Batch 2 Hamill_Democracy / Intruder 2

In [133]:
hamill_2 = create_batch(remove_dupes, 'Hamill_Democracy', 'Subpoena_Trump')
hamill_2 = pd.DataFrame.from_records(hamill_2)

In [134]:
hamill_2

,Tweets,Processed_Tweets,topic_label,intruder
0,Watching the #January6thCommitteeHearings and ...,watching seeing taking action restore order co...,Hamill_Democracy,0
1,Its hilarious that Trump is being subpoenaed f...,hilarious trump subpoenaed jan6 hearing needs ...,Hamill_Democracy,0
2,When the violent armed insurrectionists were c...,violent armed insurrectionists coming stepped ...,Hamill_Democracy,0
3,#January6thCommitteeHearings This is why Jaime...,jaime lost primary right thing needs win maga ...,Hamill_Democracy,0
4,I woke up this morning in cold sweats just rem...,woke morning cold sweats remembering happened ...,Subpoena_Trump,1


#### Batch 2 Hamill_Democracy / Intruder 3

In [135]:
hamill_3 = create_batch(remove_dupes, 'Hamill_Democracy', 'Meadows_Guiliani_Stone')
hamill_3 = pd.DataFrame.from_records(hamill_3)

In [136]:
hamill_3

,Tweets,Processed_Tweets,topic_label,intruder
0,"You've been conned, #MAGA.\nYou continue to be...",conned maga continue conned trump knew lost li...,Hamill_Democracy,0
1,@katiehobbs Trump admitted he lied and knows h...,trump admitted lied knows lost lake whole camp...,Hamill_Democracy,0
2,@TheLeoTerrell Only moral cowards would run te...,moral cowards would run terrified watching tv ...,Hamill_Democracy,0
3,RT @jossytweets: I bet Trump is gonna start a ...,bet trump gonna start new conspiracy democrats...,Meadows_Guiliani_Stone,1
4,"The future is predictable. \nBy the way, Trump...",future predictable way trump coming trump puti...,Hamill_Democracy,0


#### Batch 3 Meadows_Guiliani_Stone / Intruder 1

In [137]:
mgs_1 = create_batch(remove_dupes, 'Meadows_Guiliani_Stone', 'Mitch_Kevin_Knew')
mgs_1 = pd.DataFrame.from_records(mgs_1)

In [138]:
mgs_1

,Tweets,Processed_Tweets,topic_label,intruder
0,RT @DeanObeidallah: In no other first world na...,first world nation would donald trump able att...,Meadows_Guiliani_Stone,0
1,Nancy Pelosi is BadAss 81 at the time not Scar...,nancy pelosi badass 81 time scared corner hidi...,Meadows_Guiliani_Stone,0
2,"RT @joekatz45: Trump: ""I literally don't care ...",trump literally care live die kevin mccarthy f...,Mitch_Kevin_Knew,1
3,"RT @Jemappelleamie: Dear Donald, you lost Mich...",dear donald lost michigan pos signed michigan,Meadows_Guiliani_Stone,0
4,@FoxNews The Big Takeaway from today’s hearing...,big takeaway today hearing trump attempted cou...,Meadows_Guiliani_Stone,0


#### Batch 3 Meadows_Guiliani_Stone / Intruder 2

In [139]:
mgs_2 = create_batch(remove_dupes, 'Meadows_Guiliani_Stone', 'Subpoena_Trump')
mgs_2 = pd.DataFrame.from_records(mgs_2)

In [140]:
mgs_2

,Tweets,Processed_Tweets,topic_label,intruder
0,COMMUNISTA BLUMENTHAL FALSE CLAIM CHARGES &amp...,communista blumenthal false claim charges amp ...,Meadows_Guiliani_Stone,0
1,"RT @mayawiley: This footage. Pelosi, Schumer t...",footage pelosi schumer trying get help capitol...,Meadows_Guiliani_Stone,0
2,"Anyone else stunned to learn Pelosi, Schumer &...",anyone else stunned learn pelosi schumer amp s...,Meadows_Guiliani_Stone,0
3,It is time for Democrats to get tough on crime...,time democrats get tough crime people need get...,Meadows_Guiliani_Stone,0
4,“What kind of snowflake is Donald Trump that h...,kind snowflake donald trump willing send peopl...,Subpoena_Trump,1


#### Batch 3 Meadows_Guiliani_Stone / Intruder 3

In [141]:
mgs_3 = create_batch(remove_dupes, 'Meadows_Guiliani_Stone', 'Hamill_Democracy')
mgs_3 = pd.DataFrame.from_records(mgs_3)

In [142]:
mgs_3

,Tweets,Processed_Tweets,topic_label,intruder
0,"RT @BrennanCenter: Rep. Schiff: ""Evidence show...",rep schiff evidence shows trump aware risk vio...,Hamill_Democracy,1
1,Nancy Pelosi is BadAss 81 at the time not Scar...,nancy pelosi badass 81 time scared corner hidi...,Meadows_Guiliani_Stone,0
2,"@TomFitton 🧵NEW from me: Tom Fitton, the far-r...",new tom fitton far right fake lawyer got trump...,Meadows_Guiliani_Stone,0
3,"@SpiroAgnewGhost When you watch that video, sh...",watch video look crazy watch pretty clear craz...,Meadows_Guiliani_Stone,0
4,COMMUNISTA BLUMENTHAL VIETNAM SERVICE LIE &amp...,communista blumenthal vietnam service lie amp ...,Meadows_Guiliani_Stone,0


#### Batch 4 Subpoena Trump / Intruder 1

In [143]:
sub_tr_1 = create_batch(remove_dupes, 'Subpoena_Trump', 'Mitch_Kevin_Knew')
sub_tr_1 = pd.DataFrame.from_records(sub_tr_1)

In [144]:
sub_tr_1

,Tweets,Processed_Tweets,topic_label,intruder
0,"RT @jbf1755: Wow -- supposedly, the Committee ...",wow supposedly committee vote trump subpoena h...,Subpoena_Trump,0
1,On the same day treasonous Donald Trump was su...,day treasonous donald trump subpoenaed supreme...,Subpoena_Trump,0
2,https://t.co/SYktQBD6en\n#Jan6thInsurrection ...,jan6thinsurrection 6 committee votes subpoena ...,Subpoena_Trump,0
3,Since #Trump was subpoena there should be a 10...,since trump subpoena 10th primetime world coul...,Subpoena_Trump,0
4,@MacFarlaneNews I tried but I cannot read this...,tried cannot read nonsense revealed need know ...,Mitch_Kevin_Knew,1


In [145]:
sub_tr_2 = create_batch(remove_dupes, 'Subpoena_Trump', 'Meadows_Guiliani_Stone')
sub_tr_2 = pd.DataFrame.from_records(sub_tr_2)

In [146]:
sub_tr_2

,Tweets,Processed_Tweets,topic_label,intruder
0,RT @MmeDefargeKnits: Good #Catholic girl Nancy...,good catholic girl nancy pelosi say word shit ...,Meadows_Guiliani_Stone,1
1,https://t.co/VqPQuVsbx6\nDINOBROS President Do...,dinobros president donald trump 2020 toy figur...,Subpoena_Trump,0
2,RT @DrDinD: As the #January6thCommitteeHearing...,continue today reminder cannot election denier...,Subpoena_Trump,0
3,"RT @petenet321: ""President Donald Trump""\n#Jan...",president donald trump president trump loves i...,Subpoena_Trump,0
4,RT @d08ef2df608d4a8: So Trump tried to sabotag...,trump tried sabotage us troops immediate withd...,Subpoena_Trump,0


In [147]:
sub_tr_3 = create_batch(remove_dupes, 'Subpoena_Trump', 'Hamill_Democracy')
sub_tr_3 = pd.DataFrame.from_records(sub_tr_3)

In [148]:
sub_tr_3

,Tweets,Processed_Tweets,topic_label,intruder
0,@SpeakerPelosi is doing what Trump should have...,trump charge jan6thinsurrection jan6thhearings...,Subpoena_Trump,0
1,@DonaldJTrumpJr please pass a message on to yo...,please pass message dad lest see dad balls tes...,Subpoena_Trump,0
2,#January6thCommitteeHearings are such a stage ...,stage performance staged pelosi calls released...,Subpoena_Trump,0
3,I'd rather see baby formula on the shelves tha...,rather see baby formula shelves trump testifying,Subpoena_Trump,0
4,#January6thCommitteeHearings This is why Jaime...,jaime lost primary right thing needs win maga ...,Hamill_Democracy,1


#### All Batches Into Dataframe

In [149]:
frames = [mitch_kev_1,
mitch_kev_2,
mitch_kev_3,
hamill_1,
hamill_2,
hamill_3,
mgs_1,
mgs_2,
mgs_3,
sub_tr_1,
sub_tr_2,
sub_tr_3]

In [150]:
result = pd.concat(frames)

In [151]:
# reset index
result.reset_index(drop=True, inplace=True)

In [152]:
result.head()

,Tweets,Processed_Tweets,topic_label,intruder
0,@The_FJC That #January6thCommitteeHearings was...,fjc planned yesterday exactly reason knew repo...,Mitch_Kevin_Knew,0
1,@washingtonpost #America #AmericaFirst #Januar...,america americafirst making martyr trump maga,Hamill_Democracy,1
2,Donald Trump knew he lost.” 2/2 - Chmn. Bennie...,donald trump knew lost 2 2 chmn bennie thompso...,Mitch_Kevin_Knew,0
3,These #January6thCommitteeHearings have been e...,explosive knew jan 6 premeditated tfg amp team...,Mitch_Kevin_Knew,0
4,RT @ColorMeStevie: Mr. Trump is responsible fo...,mr trump responsible 6th period end lock,Mitch_Kevin_Knew,0


In [153]:
#create column with intruder index

In [154]:
result['intruder_index'] = None

In [155]:
# Assign the index to the new column where column 'B' is True
result.loc[result['intruder'] == True, 'intruder_index'] = result.index[result['intruder'] == True]

In [156]:
result

,Tweets,Processed_Tweets,topic_label,intruder,intruder_index
0,@The_FJC That #January6thCommitteeHearings was...,fjc planned yesterday exactly reason knew repo...,Mitch_Kevin_Knew,0,None
1,@washingtonpost #America #AmericaFirst #Januar...,america americafirst making martyr trump maga,Hamill_Democracy,1,1
2,Donald Trump knew he lost.” 2/2 - Chmn. Bennie...,donald trump knew lost 2 2 chmn bennie thompso...,Mitch_Kevin_Knew,0,None
3,These #January6thCommitteeHearings have been e...,explosive knew jan 6 premeditated tfg amp team...,Mitch_Kevin_Knew,0,None
4,RT @ColorMeStevie: Mr. Trump is responsible fo...,mr trump responsible 6th period end lock,Mitch_Kevin_Knew,0,None
5,And here is weasel Mitch McConnell …in April 2...,weasel mitch mcconnell april 2022 said would s...,Mitch_Kevin_Knew,0,None
6,On Jan 6 speaker Pelosi was President pelosi s...,jan 6 speaker pelosi president pelosi one trul...,Subpoena_Trump,1,6
7,RT @taradublinrocks: Trump knew he lost. \nTru...,trump knew lost trump team knew lost kept goin...,Mitch_Kevin_Knew,0,None
8,The committee presented all the evidence. Trum...,committee presented evidence trump lap dogs st...,Mitch_Kevin_Knew,0,None
9,FBI Director Chris Wray IS PART of the #Republ...,fbi director chris wray part republicans coup ...,Mitch_Kevin_Knew,0,None


#### CSVs

In [157]:
#result.to_csv('result_draft_final.csv')

In [85]:
#mitch_kev.to_csv('batch1_intruder1.csv', index=False)

### Test it's working as intended

The test will verify that the batch has exactly 5 documents and that only one of them is an intruder. Also, it will verify that the documents from the same topic are not intruders.
- Is this test good? How to improve
- Give you an idea of seeing how this is working

In [86]:
# import unittest
# from unittest.mock import patch
# import pytest
# import pandas as pd
# import random

# # Assuming create_batch() is in a file named my_module.py
# from my_module import create_batch

# class TestCreateBatch(unittest.TestCase):
#     @patch("random.shuffle")
#     def test_create_batch(self, mock_shuffle):
#         # Mock shuffle function to do nothing (to make the test deterministic)
#         mock_shuffle.side_effect = lambda x: x

#         # Create a mock dataframe for testing
#         data = {
#             'topic_label': ['topic1', 'topic1', 'topic1', 'topic1', 'topic1', 'topic2'],
#             'content': ['doc1', 'doc2', 'doc3', 'doc4', 'doc5', 'intruder']
#         }
#         df = pd.DataFrame(data)

#         # Use the function
#         batch = create_batch(df, 'topic1', 'topic2')

#         # Test if the batch has exactly 5 documents
#         self.assertEqual(len(batch), 5)

#         # Test if only one of them is an intruder
#         intruder_count = sum(1 for doc in batch if doc['intruder'] == 1)
#         self.assertEqual(intruder_count, 1)

#         # Test if the documents from the same topic are not intruders
#         for doc in batch:
#             if doc['topic_label'] == 'topic1':
#                 self.assertEqual(doc['intruder'], 0)

# # Run the test
# if __name__ == '__main__':
#     unittest.main()


In [87]:
#You would put this code in a file, say test_create_batch.py, and run it with the command pytest test_create_batch.py. Please note that you need to have the pytest module installed. You can install it using pip:
# pip install pytest
# If the tests pass, you can be confident that your function is working correctly. If any of the tests fail, pytest will provide detailed output that can help you identify the issue.

# import pytest

# def test_create_batch():
#     # Setup
#     df = pd.read_csv('/mnt/data/simple_df.csv')

#     # Test that the function returns a list of length 5
#     batch = create_batch(df, 'Subpoena_Trump')
#     assert len(batch) == 5

#     # Test that the function returns 4 documents with "intruder" set to 0 and 1 document with "intruder" set to 1
#     intruders = [doc['intruder'] for doc in batch]
#     assert intruders.count(0) == 4
#     assert intruders.count(1) == 1

#     # Test that the function returns 4 documents with the same topic and 1 document with a different topic
#     topics = [doc['topic'] for doc in batch]
#     assert topics.count('Subpoena_Trump') == 4
#     assert 'Subpoena_Trump' not in topics[-1]

#     # Test that the function raises a ValueError when the provided main topic is not in the dataframe
#     with pytest.raises(ValueError):
#         create_batch(df, 'Nonexistent_Topic')